In [ ]:
!pip install faker

In [ ]:
import csv
import numpy as np
import random
import math
from faker import Faker

fake = Faker()
manufacturers = ['xiaomi', 'sony', 'fitbit', 'coospo', 'huawei', 'garmin', 'apple', 'oura']
genders = ['M', 'F']
educations = ['school', 'bachelor', 'master', 'phd']
marital_statuses = ['single', 'married', 'divorced', 'separated', 'widowed']


with open('fitness_wearables.csv', 'w', newline='') as csvfile:
    fieldnames = ['manufacturer',
                  'age',
                  'training_before[Y/N]',
                  'gender',
                  'body_fat[%]',
                  'smoking[Y/N]',
                  'alcohol_times_week',
                  'occupation',
                  'education',
                  'marital_status',
                  'kids',
                  'location',
                  'N sessions/w 1m1w',
                  'N sessions/w 1m2w',
                  'N sessions/w 1m3w',
                  'N sessions/w 1y1w',
                  'N sessions/w 1y2w',
                  'N sessions/w 1y3w']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for _ in range(0, 3000):
        manufacturer = np.random.choice(manufacturers)
        # 25-45 range is 2x more likely.
        rand_ages = [random.randrange(25, 46), random.randrange(18, 26), random.randrange(45, 66)]
        age = np.random.choice(rand_ages, p=[0.5, 0.25, 0.25])
        #depends on age: empty for <22 y.o.
        if age >= 22:
            occupation = fake.job()
        else:
            occupation = ''
        #training_before[Y/N]
        training_before = fake.boolean(chance_of_getting_true=75)
        gender = np.random.choice(genders)
        #depends on training_before and gender
        #figures inspired by https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#:~:text=Measuring%20body%20fat,-Another%20option%20is&text=Men%20and%20women%20need%20different,than%2032%25%20classifies%20as%20obesity.
        #female trained before
        if gender == 'F' and training_before:
            #Creating an overlap with non-training
            body_fat = random.randrange(14, 30)
        #female NOT trained before
        elif gender == 'F' and not training_before:
            body_fat = random.randrange(18, 40)
        #male trained before
        elif gender == 'M' and training_before:
            #Creating an overlap with non-training
            body_fat = random.randrange(8, 22)
        #male NOT trained before
        else:
            body_fat = random.randrange(13, 30)
        smoking = fake.boolean(chance_of_getting_true=25)
        #alcohol
        alcohol = random.randrange(0, 4)
        #education, age dependent
        if age < 22:
            education = educations[0]
        elif age > 27:
            education = np.random.choice(educations)
        else:
            education = np.random.choice(educations[1:3])
        #marital status, age dependent
        if age < 20:
            marital_status = marital_statuses[0]
        elif 20 <= age < 40:
            marital_status = np.random.choice(marital_statuses[:4])
        else:
            marital_status = np.random.choice(marital_statuses)
        #kids, age dependent
        if age < 22:
            kids = 0
        elif age < 30:
            kids = random.randrange(0, 3)
        else:
            kids = random.randrange(0, 4)
        #location
        location = fake.address()
    
        min = random.randrange(2, 4)
        max = random.randrange(4, 7)
        n_sessions_1m1w  = random.randrange(min, max)
        n_sessions_1m2w = random.randrange(min, max)
        n_sessions_1m3w = random.randrange(min, max)
        rand_ages = [random.randrange(25, 46), random.randrange(18, 26), random.randrange(45, 66)]
        weight = 0.2
        coef = (1 - weight*(0 if 25 <= age <= 45 else 1)
            - weight*(not training_before)
            - weight*(smoking)
            - weight*(0 if alcohol <=2 else 1)
            - weight*(0 if (gender == 'F' and body_fat <=25) or (gender == 'M' and body_fat <=18) else 1))
        n_sessions_1y1w = math.ceil(n_sessions_1m1w * coef)
        n_sessions_1y2w = math.ceil(n_sessions_1m2w * coef)
        n_sessions_1y3w = math.ceil(n_sessions_1m3w * coef)
    
        writer.writerow({
            'manufacturer': manufacturer,
            'age': age,
            'training_before[Y/N]': training_before,
            'gender': gender,
            'body_fat[%]': body_fat,
            'smoking[Y/N]': smoking,
            'alcohol_times_week': alcohol,
            'occupation': occupation,
            'education': education,
            'marital_status': marital_status,
            'kids': kids,
            'location': location,
            'N sessions/w 1m1w': n_sessions_1m1w,
            'N sessions/w 1m2w': n_sessions_1m2w,
            'N sessions/w 1m3w': n_sessions_1m3w,
            'N sessions/w 1y1w': n_sessions_1y1w,
            'N sessions/w 1y2w': n_sessions_1y2w,
            'N sessions/w 1y3w': n_sessions_1y3w})
